In [103]:
# import libraries
import subprocess
import json
import os
from constants import *
from dotenv import load_dotenv
from bipwallet import wallet
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

load_dotenv()

True

In [104]:
# import mnemonic from env
filepath="env.bat"
p = subprocess.Popen(filepath, shell=True, stdout = subprocess.PIPE)

stdout, stderr = p.communicate()
print (p.returncode) # is 0 if success
mnemonic = os.getenv('MNEMONIC')
print(mnemonic)

1
cannon chronic machine excess hope blanket giraffe cancel behave hello absurd input


In [105]:
# connect to local ETH/ geth
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [119]:
# Define function to derive wallet
def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys

In [125]:
# Test the function derive_wallets
derive_wallets(mnemonic, 'BTC', 3)

[{'path': "m/44'/0'/0'/0/0",
  'address': '16TbGXrr2gjCAkSnxb1dseACLUEgb1FUYQ',
  'xprv': 'xprvA3xaAvWDJc2B8mPFyEMzQotCRDLah4vn7T5toQ7LiFSEAjqjvUt3gQ5ztaDcrNfeXuSoRjdfQufghFgjHJWtXxDbpQZqFWrRWhyjHfM3SsU',
  'xpub': 'xpub6GwvaS378yaUMFTj5FtzmwpvyFB56XedUg1VbnWxGayD3YAtU2CJECQUjsnMTcgfmy7GmSxhU3rRUp3cmMkzdx9dmjkeVHphVrhLqTSp1iq',
  'privkey': 'KyM3bVtH4Kg1Q4CitFNJSf4rvtqhdZAkyTuALBjyZaioZw3t9W7g',
  'pubkey': '038d285ea71e7c2dd48b0d93bca5db5b40cc05eede41afe894c5e29415fb6d6fc6',
  'pubkeyhash': '3be00c9af3c214fcf0acf871a59733ca8bd9b636',
  'index': 0},
 {'path': "m/44'/0'/0'/0/1",
  'address': '1LNvZPxNiW3xymMg8mMzVbiEaRCN68KvB5',
  'xprv': 'xprvA3xaAvWDJc2BAgXZC947voTrRGp6gVtEt7KXEVqWs7XGsAeSugMjKp4PHmMn2PSe4Boc1mRFnvRyHWcunjt7E86Pa4FG8hHpVfqG3M38zfD',
  'xpub': 'xpub6GwvaS378yaUPAc2JAb8HwQayJeb5xc6FLF82tF8RT4FjxybTDfyscNs92K86SZ2nNKZPFQcbC7uQ864SurCaMWbix17C7MHRcxKCaqK1v5',
  'privkey': 'KyN3SXH8dHgqTSu1UJ2uLVBCkTG63JBSE5XoN7GozsYR4vYTv94i',
  'pubkey': '0235eca9c4d08c9fc3424b753e8be682

In [120]:
#Setting dictionary of coins to be used in the wallet
coins = {"eth", "btc-test", "btc"}
numderive = 3

In [127]:
# 
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(mnemonic, coin, numderive=3)

In [128]:

eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']

print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))

"0x30c2577db89760baa9ba4058b1033b9e103f287e5de1689e35833ee8e7a7c857"
"cNYVznjLVmB5XANVr4SJ6choENNntn1t96jPQbn51rGUjTrpsnV7"


In [129]:
# create a function that convert the privkey string in a child key to an account object.
def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)



In [130]:
def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [138]:
# create a function to hold Ethereum 
eth_acc = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH,5)[0]['privkey'])


eth
0x30c2577db89760baa9ba4058b1033b9e103f287e5de1689e35833ee8e7a7c857


In [115]:
# create a function to send txn
def send_txn(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        signed_txn = account.sign_transaction(txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

In [133]:
#use the send_txn function to send transactions 
send_txn(ETH,eth_acc,'0x46BbdBf56ff911A93AdaF0164d0709F78B52765E',1)

'0x4f8b0c162957811ddb7deb1b297fa707e41a81e65def03d89659fb6ad0a2184f'

## BTC-TEST Transactions

In [144]:
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

btc-test
cNYVznjLVmB5XANVr4SJ6choENNntn1t96jPQbn51rGUjTrpsnV7


In [149]:
# create BTC transaction
create_tx(BTCTEST,btc_acc,"mtK73sNPY9CKuzVvpv4W1969AD1UmGGfsX", 0.1)

'{"unspents":[{"amount":49808912,"confirmations":-1781555,"script":"76a914edf426ec881075b8ad83bd43951d86ab79007d3988ac","txid":"62989ab0fa72c9ed0a2de95e878d94391522a036a448ad4bbeeaa2949fed2b8e","txindex":14,"type":"p2pkh","vsize":148,"segwit":false},{"amount":100000,"confirmations":-1781480,"script":"76a914edf426ec881075b8ad83bd43951d86ab79007d3988ac","txid":"f509aa70dd53d1d1938d6c3916906e1c49183a01e199fafc07fa20bce31f57f7","txindex":0,"type":"p2pkh","vsize":148,"segwit":false},{"amount":23000,"confirmations":-1781480,"script":"76a914edf426ec881075b8ad83bd43951d86ab79007d3988ac","txid":"d5c99494f1dabcc740edd4126430e9baa09fdd09a30400830f3906245dfd45ff","txindex":0,"type":"p2pkh","vsize":148,"segwit":false},{"amount":10969,"confirmations":-1781479,"script":"76a914edf426ec881075b8ad83bd43951d86ab79007d3988ac","txid":"d117e459d8ce4eec6ff607fd7002790372f0a7c6288ae1ebfffbb3a1133ddff0","txindex":1,"type":"p2pkh","vsize":148,"segwit":false}],"outputs":[["mtK73sNPY9CKuzVvpv4W1969AD1UmGGfsX",100

In [151]:
# Send BTC transaction
create_tx(BTCTEST,btc_acc,"mtK73sNPY9CKuzVvpv4W1969AD1UmGGfsX", 0.1)

'{"unspents":[{"amount":49808912,"confirmations":-1781555,"script":"76a914edf426ec881075b8ad83bd43951d86ab79007d3988ac","txid":"62989ab0fa72c9ed0a2de95e878d94391522a036a448ad4bbeeaa2949fed2b8e","txindex":14,"type":"p2pkh","vsize":148,"segwit":false},{"amount":100000,"confirmations":-1781480,"script":"76a914edf426ec881075b8ad83bd43951d86ab79007d3988ac","txid":"f509aa70dd53d1d1938d6c3916906e1c49183a01e199fafc07fa20bce31f57f7","txindex":0,"type":"p2pkh","vsize":148,"segwit":false},{"amount":23000,"confirmations":-1781480,"script":"76a914edf426ec881075b8ad83bd43951d86ab79007d3988ac","txid":"d5c99494f1dabcc740edd4126430e9baa09fdd09a30400830f3906245dfd45ff","txindex":0,"type":"p2pkh","vsize":148,"segwit":false},{"amount":10969,"confirmations":-1781479,"script":"76a914edf426ec881075b8ad83bd43951d86ab79007d3988ac","txid":"d117e459d8ce4eec6ff607fd7002790372f0a7c6288ae1ebfffbb3a1133ddff0","txindex":1,"type":"p2pkh","vsize":148,"segwit":false}],"outputs":[["mtK73sNPY9CKuzVvpv4W1969AD1UmGGfsX",100

## ETH Transactions

In [147]:
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

ValueError: You can't add the same un-named instance twice